In [1]:
from google import colab
colab.drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -r requirements.txt

In [2]:

"""
3GPP DOCX Document Processing Pipeline
Comprehensive system to parse 3GPP specification documents (DOCX format)
and structure data for knowledge graph construction
"""


import re
import json
import mammoth
from bs4 import BeautifulSoup
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional
from datetime import datetime

In [ ]:
@dataclass
class DocumentMetaData:
    specification_id: str
    version: str
    title: str
    file_path: str

@dataclass
class SectionStructure:
    section_id: str
    title: str
    level: int
    parent_section: Optional[str] = None
    content: str = ""
    tables: List[Dict] = None

@dataclass
class ProcessedChunk:
    chunk_id: str
    section_id: str
    section_title: str
    content: str
    chunk_type: str
    cross_references: List[str]
    tables: List[Dict] = None


In [14]:
class ThreeGPPParser:
    def __init__(self):
        self.section_pattern = re.compile(r'^(\d+(?:\.\d+)*)\s+(.+)$')

        # Basic reference patterns
        self.clause_pattern = re.compile(r'(?:clause|section)\s+(\d+(?:\.\d+)*)', re.IGNORECASE)
        self.table_pattern = re.compile(r'table\s+(\d+(?:\.\d+)*(?:-\d+)?)', re.IGNORECASE)
        self.figure_pattern = re.compile(r'figure\s+(\d+(?:\.\d+)*(?:-\d+)?)', re.IGNORECASE)

        # External reference patterns
        self.external_patterns = [
            # "TS 38.331, clause 4.1"
            re.compile(r'(?:3GPP\s+)?(?:TS|TR)\s+(\d+\.\d+)(?:,?\s+(?:clause|section)\s+(\d+(?:\.\d+)*))', re.IGNORECASE),
            # "clause 4.1 of TS 38.331"
            re.compile(r'(?:clause|section)\s+(\d+(?:\.\d+)*)\s+of\s+(?:3GPP\s+)?(?:TS|TR)\s+(\d+\.\d+)', re.IGNORECASE),
            # "table 4.1 of TS 38.331"
            re.compile(r'table\s+(\d+(?:\.\d+)*(?:-\d+)?)\s+of\s+(?:3GPP\s+)?(?:TS|TR)\s+(\d+\.\d+)', re.IGNORECASE),
            # "figure 4.1 of TS 38.331"
            re.compile(r'figure\s+(\d+(?:\.\d+)*(?:-\d+)?)\s+of\s+(?:3GPP\s+)?(?:TS|TR)\s+(\d+\.\d+)', re.IGNORECASE),
            # Standalone TS
            re.compile(r'(?:3GPP\s+)?(?:TS|TR)\s+(\d+\.\d+)(?!\s*(?:,?\s*(?:clause|section|table|figure)))', re.IGNORECASE)
        ]

    def extract_metadata(self, document_path: str) -> DocumentMetaData:
        """Extract basic metadata from document"""
        with open(document_path, "rb") as docx_file:
            result = mammoth.convert_to_html(docx_file)

        soup = BeautifulSoup(result.value, 'html.parser')
        text = soup.get_text()

        # Find TS number and version from document content
        # Pattern: "3GPP TS 23.501 V18.9.0 (2025-03)"
        ts_version_match = re.search(r'3GPP\s+TS\s+(\d+\.\d+)\s+V(\d+\.\d+\.\d+)', text)

        if ts_version_match:
            ts_number = ts_version_match.group(1)  # 23.501
            version = ts_version_match.group(2)    # 18.9.0
            specification_id = f"ts_{ts_number}"   # ts_23.501
            title = f"3GPP TS {ts_number}"
        else:
            # Fallback: extract from filename
            filename = Path(document_path).stem
            filename_match = re.search(r'(\d{2})(\d{3})', filename)
            if filename_match:
                ts_number = f"{filename_match.group(1)}.{filename_match.group(2)}"
                specification_id = f"ts_{ts_number}"
                version = "Unknown"
                title = f"3GPP TS {ts_number}"
            else:
                specification_id = "ts_unknown"
                version = "Unknown"
                title = "Unknown Document"

        return DocumentMetaData(
            specification_id=specification_id,  # ts_23.501 format
            version=version,                     # 18.9.0 format
            title=title,                        # 3GPP TS 23.501
            file_path=str(document_path)
        )

    def parse_sections(self, document_path: str) -> List[SectionStructure]:
        """Parse document sections with content"""
        with open(document_path, "rb") as docx_file:
            result = mammoth.convert_to_html(docx_file)

        soup = BeautifulSoup(result.value, 'html.parser')
        sections = []
        current_section = None
        in_content = False

        for element in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'table']):
            text = element.get_text().strip()
            if element.name == 'table':
                if current_section:
                    table_data = {
                        "headers" : [],
                        "rows" : []
                    }
                    rows = element.find_all('tr')
                    if(rows):
                        # first row as headers
                        headers = [th.get_text().strip() for th in rows[0].find_all(['th', 'td'])]
                        table_data["headers"] = headers

                        # the rest as data rows
                        for row in rows[1:]:
                            row_data = [td.get_text().strip() for td in row.find_all(['td', 'th'])]
                            if row_data:
                                table_data["rows"].append(row_data)
                    if current_section.tables is None:
                        current_section.tables = []
                    current_section.tables.append(table_data)
                continue
            if not text:
                continue

            # Start content after first h1
            if element.name == 'h1' and not in_content:
                in_content = True
            if not in_content:
                continue

            # Check if it's a section heading
            if element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                match = self.section_pattern.match(text)
                if match:
                    section_number = match.group(1)
                    section_title = match.group(2)
                    level = len(section_number.split('.'))
                    parent = '.'.join(section_number.split('.')[:-1]) if level > 1 else None

                    current_section = SectionStructure(
                        section_id=section_number,
                        title=section_title,
                        level=level,
                        parent_section=parent,
                        content="",
                        tables=[]
                    )
                    sections.append(current_section)
                    continue

            # Add content to current section
            if current_section:
                if current_section.content:
                    current_section.content += '\n' + text
                else:
                    current_section.content = text

        return sections

    def is_external_ref(self, text: str, match_start: int, match_end: int) -> bool:
        """Check if reference has nearby TS/TR"""
        window = 30
        context = text[max(0, match_start-window):match_end+window]
        return bool(re.search(r'(?:3GPP\s+)?(?:TS|TR)\s+\d+\.\d+', context, re.IGNORECASE))

    def extract_cross_references(self, content: str) -> List[str]:
        """Extract cross-references from content"""
        cross_refs = []

        # Process basic references (clause, table, figure)
        for pattern, ref_type in [(self.clause_pattern, 'clause'),
                                 (self.table_pattern, 'table'),
                                 (self.figure_pattern, 'figure')]:
            for match in pattern.finditer(content):
                ref_num = match.group(1)

                if self.is_external_ref(content, match.start(), match.end()):
                    # Try to find TS number nearby
                    start = max(0, match.start() - 50)
                    end = min(len(content), match.end() + 50)
                    context = content[start:end]

                    ts_match = re.search(r'(?:3GPP\s+)?(?:TS|TR)\s+(\d+\.\d+)', context, re.IGNORECASE)
                    if ts_match:
                        spec_num = ts_match.group(1)
                        cross_refs.append(f"ts_{spec_num}_{ref_type}_{ref_num}")
                    else:
                        cross_refs.append(f"external_{ref_type}_{ref_num}")
                else:
                    cross_refs.append(f"{ref_type}_{ref_num}")

        # Process explicit external references
        for pattern in self.external_patterns:
            for match in pattern.finditer(content):
                groups = match.groups()

                if len(groups) == 1:  # Standalone TS
                    cross_refs.append(f"ts_{groups[0]}")
                elif len(groups) == 2:
                    # Determine format and extract spec + item
                    if 'of' in match.group(0):  # "item X of TS Y" format
                        item_num, spec_num = groups
                        if 'clause' in match.group(0):
                            cross_refs.append(f"ts_{spec_num}_clause_{item_num}")
                        elif 'table' in match.group(0):
                            cross_refs.append(f"ts_{spec_num}_table_{item_num}")
                        elif 'figure' in match.group(0):
                            cross_refs.append(f"ts_{spec_num}_figure_{item_num}")
                    else:  # "TS Y, item X" format
                        spec_num, item_num = groups
                        if 'clause' in match.group(0):
                            cross_refs.append(f"ts_{spec_num}_clause_{item_num}")

        # Remove duplicates
        return list(dict.fromkeys(cross_refs))

    def classify_content_type(self, title: str, content: str) -> str:
        """Simple content classification"""
        title_lower = title.lower()
        content_lower = content.lower()

        if any(term in title_lower for term in ['definition', 'overview', 'general']):
            return 'definition'
        elif any(term in title_lower for term in ['procedure', 'flow', 'process']):
            return 'procedure'
        elif any(term in title_lower for term in ['parameter', 'identifier']):
            return 'parameter'
        elif 'shall' in content_lower[:200]:  # Check first 200 chars
            return 'requirement'
        else:
            return 'general'

    def create_chunks(self, sections: List[SectionStructure], metadata: DocumentMetaData) -> List[ProcessedChunk]:
        """Create content chunks"""
        chunks = []

        for section in sections:
            if not section.content.strip():
                continue

            chunk = ProcessedChunk(
                chunk_id=f"{metadata.specification_id}_{section.section_id}",
                section_id=section.section_id,
                section_title=section.title,
                content=section.content,
                chunk_type=self.classify_content_type(section.title, section.content),
                cross_references=self.extract_cross_references(section.content),
                tables=section.tables
            )
            chunks.append(chunk)

        return chunks

    def save_to_json(self, chunks: List[ProcessedChunk], metadata: DocumentMetaData, output_path: str = None):
        """Save chunks to JSON file"""
        if output_path is None:
            output_path = f"{metadata.specification_id}_.json"

        data = {
            "metadata": asdict(metadata),
            "export_info": {
                "export_date": datetime.now().isoformat(),
                "total_chunks": len(chunks)
            },
            "chunks": [asdict(chunk) for chunk in chunks]
        }

        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"✅ Saved {len(chunks)} chunks to {output_path}")
        return output_path


In [15]:
def process_document(document_path: str, output_path: str):
    """Process a 3GPP document and return chunks"""
    parser = ThreeGPPParser()

    # Extract metadata
    metadata = parser.extract_metadata(document_path)
    print(f"📄 Processing: {metadata.title}")

    # Parse sections
    sections = parser.parse_sections(document_path)
    print(f"📂 Found {len(sections)} sections")

    # Create chunks
    chunks = parser.create_chunks(sections, metadata)
    print(f"📦 Created {len(chunks)} chunks")
    output_path = output_path + metadata.specification_id + ".json"
    # Save to JSON
    output_file = parser.save_to_json(chunks, metadata, output_path)

    return chunks, metadata, output_file


In [16]:
# EXAMPLE FOR 1 doc
document_path = "/home/linguyen/3GPP/document_processing/data/29_series/29500-i80/29500-i80.docx"
chunks, metadata, output_file = process_document(document_path, "")

# # Show first few chunks
# print("\n📋 Sample chunks:")
# for chunk in chunks[:5]:
#     print(f"  {chunk.chunk_id}: {chunk.section_title}")
#     print(f"    Type: {chunk.chunk_type}")
#     print(f"    Cross-refs: {chunk.cross_references[:3]}...")  # Show first 3
#     print()

📄 Processing: 3GPP TS 29.500
📂 Found 238 sections
📦 Created 183 chunks
✅ Saved 183 chunks to ts_29.500.json


In [ ]:
# # download 3gpp document
# # only run if you haven't have it yet
# !pip install download_3gpp
# !download_3gpp --rel 18
# !cp -r /content/Rel-18/* /content/drive/MyDrive/RAG_thesis/data/

In [21]:
import os
import zipfile
import glob
import re
from pathlib import Path

def batch_process_all(data_dir: str, output_dir: str = None):
    """Process all 3GPP documents in one go"""

    # Setup
    if output_dir is None:
        output_dir = os.path.join(data_dir, "processed_json/")
    os.makedirs(output_dir, exist_ok=True)

    # Regex to identify main body files
    main_body_pattern = re.compile(r'main.*body|_1_|s\d+_s\d+', re.IGNORECASE)
    exclude_pattern = re.compile(r'cover|annex|history|amendment', re.IGNORECASE)

    # Find all DOCX files
    docx_files = []

    # Direct DOCX
    for docx_path in glob.glob(os.path.join(data_dir, "**", "*.docx"), recursive=True):
        filename = os.path.basename(docx_path)
        if main_body_pattern.search(filename) and not exclude_pattern.search(filename):
            docx_files.append(docx_path)

    # Extract ZIP files
    for zip_path in glob.glob(os.path.join(data_dir, "**", "*.zip"), recursive=True):
        try:
            extract_dir = str(Path(zip_path).parent / Path(zip_path).stem)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                for file_info in zip_ref.filelist:
                    if (file_info.filename.endswith('.docx') and
                        not file_info.filename.startswith('~') and
                        main_body_pattern.search(file_info.filename) and
                        not exclude_pattern.search(file_info.filename)):

                        zip_ref.extract(file_info.filename, extract_dir)
                        docx_files.append(os.path.join(extract_dir, file_info.filename))
        except:
            pass

    print(f"📄 Found {len(docx_files)} main body DOCX files")

    # Process all files
    success = 0
    for i, docx_path in enumerate(docx_files, 1):
        try:
            print(f"[{i}/{len(docx_files)}] {os.path.basename(docx_path)}")
            process_document(docx_path, output_dir)
            success += 1
        except Exception as e:
            print(f"❌ {e}")

    print(f"✅ Done: {success}/{len(docx_files)}")

batch_process_all("/content/drive/MyDrive/RAG_thesis/data")

📄 Found 615 DOCX files
[1/615] 29994-i00.docx
📄 Processing: 3GPP TS 29.994
📂 Found 57 sections
📦 Created 45 chunks
✅ Saved 45 chunks to /content/drive/MyDrive/RAG_thesis/data/processed_json/ts_29.994.json
[2/615] 29550-i30.docx
📄 Processing: 3GPP TS 29.550
📂 Found 64 sections
📦 Created 43 chunks
✅ Saved 43 chunks to /content/drive/MyDrive/RAG_thesis/data/processed_json/ts_29.550.json
[3/615] 29935-i00.docx
📄 Processing: 3GPP TS 29.935
📂 Found 73 sections
📦 Created 56 chunks
✅ Saved 56 chunks to /content/drive/MyDrive/RAG_thesis/data/processed_json/ts_29.935.json
[4/615] 29574-i80.docx
📄 Processing: 3GPP TS 29.574
📂 Found 196 sections
📦 Created 131 chunks
✅ Saved 131 chunks to /content/drive/MyDrive/RAG_thesis/data/processed_json/ts_29.574.json
[5/615] 29577-i40.docx
📄 Processing: 3GPP TS 29.577
📂 Found 110 sections
📦 Created 77 chunks
✅ Saved 77 chunks to /content/drive/MyDrive/RAG_thesis/data/processed_json/ts_29.577.json
[6/615] 29555-i50.docx
📄 Processing: 3GPP TS 29.555
📂 Found 123